## TransNeo/AlphaNeo
- AlphaNeo using pre-trained TransE embeddings (optional)
- Stage: Cambrian
- Version: Spriggina
- Update Logs
    - 0713: with DeepPath style rollback at training
    - **0716: new learning paradigm, see memo for details**

#### Related Commands
- tensorboard --logdir runs
- nohup jupyter lab > jupyter.log &

In [1]:
import logging 
logging.basicConfig(level=logging.CRITICAL)

In [2]:
import os
import itertools
import copy
import random
import pickle
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

# from tensorboardX import SummaryWriter

use_cuda = torch.cuda.is_available()
print("use_cuda: {}".format(use_cuda))

use_cuda: True


In [3]:
import tyrell.spec as S
from tyrell.decider import Example

# Morpheus Version
from MorpheusInterpreter import *
from ProgramSpace import *

In [4]:
torch.__version__

'1.0.0'

In [5]:
class TransNeo(nn.Module):
    def __init__(self, p_config=None):
        super(TransNeo, self).__init__()
        self.config = p_config
        
        # predict a fixed number of shells
#         self.policy = nn.Linear(
#             self.config["embd_dim"],
#             self.config["fn"]["vocab_size"],
#         )
        
        # deeper
        self.policy0 = nn.Linear(
            self.config["embd_dim"],
            128,
        )
        self.policy1 = nn.Linear(
            128,
            self.config["fn"]["vocab_size"],
        )
        
    def forward(self, p_mapin, p_mapout):
        # p_mapin/p_mapout: (B, 15*3)
        v_delta = p_mapout-p_mapin
#         tmp_out = torch.log_softmax(
#             self.policy(v_delta),dim=1
#         )
        tmp_out = torch.log_softmax(
            self.policy1(
                F.relu(
                    self.policy0(
                        v_delta
                    )
                )
            ),dim=1
        )
        
        return tmp_out

In [6]:
class Ranker(nn.Module):
    def __init__(self, p_config=None):
        super(Ranker, self).__init__()
        self.config = p_config
        self.value_embedding = nn.Embedding(
            self.config["ranker"]["vocab_size"],
            self.config["ranker"]["embd_dim"],
        )
        self.fc0 = nn.Linear(
            self.config["ranker"]["embd_dim"] * self.config["ranker"]["max_length"] * 2,
            4096,
        )
        self.fc1 = nn.Linear(
            4096,
            2048,
        )
        self.fc2 = nn.Linear(
            2048,
            128
        )
        self.fc3 = nn.Linear(
            128,
            2
        )
        
    def forward(self, pA, pB, pC, pD):
        # pA/pB/pC/pD: (B=1, seq)
        B = pA.shape[0]
        
        # (B, seq, embd_dim)
        vA = self.value_embedding(pA).view(B, self.config["ranker"]["max_length"]*self.config["ranker"]["embd_dim"])
        vB = self.value_embedding(pB).view(B, self.config["ranker"]["max_length"]*self.config["ranker"]["embd_dim"])
        vC = self.value_embedding(pC).view(B, self.config["ranker"]["max_length"]*self.config["ranker"]["embd_dim"])
        vD = self.value_embedding(pD).view(B, self.config["ranker"]["max_length"]*self.config["ranker"]["embd_dim"])
        
        # (B=2, embd_dim * 2)
        vPOS = torch.cat(
            [
                torch.cat([vA,vB],dim=1),
                torch.cat([vA,vC],dim=1),
            ],dim=0
        )
        
        # (B=10, embd_dim * 2)
        vNEG = torch.cat(
            [
                torch.cat([vB,vA],dim=1),
                torch.cat([vB,vC],dim=1),
                torch.cat([vC,vA],dim=1),
                torch.cat([vC,vB],dim=1),
            ],dim=0
        )
        
        # (B=4*3=12, embd_dim * 2)
        vII = torch.cat([vPOS,vNEG],dim=0)
        
        v00 = F.relu(self.fc0(vII))
        v01 = F.relu(self.fc1(v00))
        v02 = F.relu(self.fc2(v01))
        v03 = self.fc3(v02)
        # == Notice: don't do any activation at the last layer ==
        # (B=4*3=12, 2)
        
        return v03
    
    def inference(self, pX, pY):
        # pX/pY: (B=1, seq)
        B = pX.shape[0]
        
        # (B, seq, embd_dim)
        vX = self.value_embedding(pX).view(B, self.config["ranker"]["max_length"]*self.config["ranker"]["embd_dim"])
        vY = self.value_embedding(pY).view(B, self.config["ranker"]["max_length"]*self.config["ranker"]["embd_dim"])
        vIN = torch.cat([vX,vY],dim=1) # (B, 2*seq*embd_dim)
        v00 = F.relu(self.fc0(vIN))
        v01 = F.relu(self.fc1(v00))
        v02 = F.relu(self.fc2(v01))
        v03 = F.softmax(self.fc3(v02),dim=1) # (B, 2)
        return v03[:,1].detach() # (B, 1)

In [7]:
# replace certain node id with certain value
def modify_shell(p_shell, p_id_from, p_id_to):
    d_prod = p_shell[0]
    d_rhs = p_shell[1]
    ld_rhs = [p_id_to if d_rhs[i]==p_id_from else d_rhs[i]
             for i in range(len(d_rhs))]
    return (d_prod, tuple(ld_rhs))

In [8]:
def compare_sketch(ps0,ps1):
    if len(ps0.node_list)!=len(ps1.node_list):
        return False
    for i in range(len(ps0.shells)):
        if ps0.node_list[-i-1].name != ps1.node_list[-i-1].name:
            return False
    return True

'''
meta-test an agent, directly run into testing / online adaptation
'''
def MetaTest(p_config, p_spec, p_interpreter, p_generator, p_model, p_bmrks, p_optim, p_writer):
    print("# Start Meta-Test...")
    
    n_solved = 0 # track the number of solved problem
    n_sketch_solved = 0
    n_attempt_list = [] # track the number of attempts in every episode
    n_sketch_atlist = [] # track the number of attempts when hitting the sketch
    
    for d_episode in range(p_config["meta_test"]["n_episode"]):
        
        # retrieve the given meta-trained model for testing
        test_model = copy.deepcopy(p_model)
        test_model.train()
        test_optim = torch.optim.Adam(list(test_model.parameters()))
        
        # ==== prepare the benchmark ====
        bmrk_prog, bmrk_str_example = p_bmrks[d_episode]
        bmrk_example = Example(
            input=[p_interpreter.load_data_into_var(p) for p in bmrk_str_example.input],
            output=p_interpreter.load_data_into_var(bmrk_str_example.output),
        )
        ps_solution = ProgramSpace(
            p_spec, p_interpreter, bmrk_example.input, bmrk_example.output,
        )
        ps_solution.init_by_prog(bmrk_prog)
        solution_prod_names = [
            ps_solution.prod_list[p[0]].name for p in ps_solution.shells
        ]
        solution_shells = ps_solution.shells
        
        # solution self-check
        if ps_solution.check_eq() is None:
            print("ERROR, SOLUTION NOT CONSISTENT!")
        
        
#         f = open("./outputs/Sarah3/Problem_{}.txt".format(d_episode), "w")
#         f.write("# Problem: {}\n\n".format(str(ps_solution.node_list[-1])))
#         f.write("# Input:\n{}\n".format(p_interpreter.renv(ps_solution.inputs[0])))
#         f.write("# Output:\n{}\n".format(p_interpreter.renv(ps_solution.output)))
#         f.flush()
        
        is_solved = False
        is_sketch_solved = False
        
        for d_attempt in range(p_config["meta_test"]["maxn_attempt"]):
            
            current_prod_names = []
            current_shells = []
            current_outputs = []
            
            selected_neurons = []
            stored_groups = [] # with lists of neurons of the same production name
            
            # in every new attempt, initialize a new Program Space
            ps_current = ProgramSpace(
                p_spec, p_interpreter, ps_solution.inputs, ps_solution.output,
            )
            # then initialize a shell template
            tmp_shell_list = ps_current.get_neighboring_shells()
            tmp_node_to_replace = ps_current.node_dict["ParamNode"][0] # for chain only
            # replace the Param Node id in shells with -1 to make them templates
            template_list = [
                modify_shell(tmp_shell_list[i],tmp_node_to_replace,-1)
                for i in range(len(tmp_shell_list))
            ]
                
            d_step = 0
            while d_step<p_config["meta_test"]["maxn_step"]:
                
                # print the training progress
                print("\r# AC/SK/EP:{}/{}/{}, AT:{}, SP:{}, att.ske.:{:.2f}, att.prog.:{:.2f},".format(
                    n_solved, n_sketch_solved, d_episode, d_attempt, d_step,
                    sum(n_sketch_atlist)/len(n_sketch_atlist) if len(n_sketch_atlist)>0 else -1,
                    sum(n_attempt_list)/len(n_attempt_list) if len(n_attempt_list)>0 else -1,
                ),end="")
                
                # ### assume chain execution, so only 1 possible returns
                # ### at d_step=0, this should be input[0]
                id_current = ps_current.get_strict_frontiers()[0]
                var_current = ps_current.node_list[id_current].ps_data # need the real var name in r env
                var_output = ps_current.output
                
                map_current = p_interpreter.camb_get_shash_abs(var_current)
                map_output = p_interpreter.camb_get_shash_abs(var_output)
                
                # make current shell list
                current_shell_list = [
                    modify_shell(template_list[i],-1,id_current)
                    for i in range(len(template_list))
                ]
                
                # wrap in B=1
                if use_cuda:
                    td_current = Variable(torch.tensor([map_current],dtype=torch.float)).cuda()
                    td_output = Variable(torch.tensor([map_output],dtype=torch.float)).cuda()
                else:
                    td_current = Variable(torch.tensor([map_current],dtype=torch.float))
                    td_output = Variable(torch.tensor([map_output],dtype=torch.float))
                    
                # (B=1, fn_vocab_size)
                td_pred = test_model(td_current, td_output)
                
                # no hints
                if random.random()<=p_config["meta_test"]["exploration_rate"]:
                    # exploration
                    tmp_id = random.choice(range(len(current_shell_list)))
                else:
                    # exploitation
                    # tmp_id = torch.multinomial(td_pred.exp().flatten(), 1).cpu().flatten().numpy()[0]
                    tmp_id = torch.argmax(td_pred.flatten()).cpu().tolist()
                    
                    
                # == Yorgia ==
                # find out all other shells that share the same product name
                tmp_component_name = ps_current.prod_list[current_shell_list[tmp_id][0]].name
                tmp_group = []
                for i in range(len(current_shell_list)):
                    if ps_current.prod_list[current_shell_list[i][0]].name==tmp_component_name:
                        tmp_group.append(td_pred[0,i])
                stored_groups.append(tmp_group)
                    
                # == Yorgia ==
                # add prod names first
                current_prod_names.append(tmp_component_name)
                # then add shells
                current_shells.append(current_shell_list[tmp_id])
                
                # update ps_current
                update_status = ps_current.add_neighboring_shell(
                    current_shell_list[tmp_id]
                )
                
                if update_status:
                    # record selected neuron
                    selected_neurons.append((True, td_pred[0,tmp_id]))
                    current_outputs.append(ps_current.node_list[-1].ps_data)
                    d_step += 1
                    
                    # succeed
                    if ps_current.check_eq() is not None:
                        # and solved!
                        is_solved = True
                        n_solved += 1
                        break
                else:
                    selected_neurons.append((False, td_pred[0,tmp_id]))
                    break
            
            # <END_FOR_STEP>
#             f.write("# ({}) Proposed {}/{}: {}\n".format(
#                 "accept" if is_solved else "reject",
#                 d_attempt, p_config["meta_test"]["maxn_attempt"],
#                 str(ps_current.node_list[-1])
#             ))
#             f.flush()
            
            if not is_sketch_solved:
                if compare_sketch(ps_current, ps_solution):
                    n_sketch_atlist.append(d_attempt)
                    is_sketch_solved = True
                    n_sketch_solved += 1
                    
            if is_solved:
                n_attempt_list.append(d_attempt)
                break
            
            # print("# Current Sketch: {}".format(current_prod_names))
            # == Yorgia ==
            # compute the loss according to the loss computation rules
            # first component, then function call
            batch_loss_list = []
            for i in range(len(current_prod_names)):
                batch_loss_list.append(
                    (-1.0)*(-selected_neurons[i][1])
                )
#                 if solution_prod_names[i]==current_prod_names[i]:
#                     # component match, promote the whole group
#                     for j in range(len(stored_groups[i])):
#                         batch_loss_list.append(
#                             (+1.0)*(-stored_groups[i][j])
#                         )
#                 else:
#                     for j in range(len(stored_groups[i])):
#                         batch_loss_list.append(
#                             (-1.0)*(-stored_groups[i][j])
#                         )
#                 # then compare function call
#                 if solution_shells[i]==current_shells[i]:
#                     batch_loss_list.append(
#                         (+1.0)*(-selected_neurons[i][1])
#                     )
#                 else:
#                     batch_loss_list.append(
#                         (-1.0)*(-selected_neurons[i][1])
#                     )
            
            batch_loss = sum(batch_loss_list)
            test_optim.zero_grad()
            batch_loss.backward()
            test_optim.step()
            
                
        # <END_FOR_ATTEMPT>     
        
#         f.close()
            
    # <END_FOR_EPISODE>
    

In [9]:
m_interpreter = MorpheusInterpreter()
m_spec = S.parse_file('./example/camb3.tyrell')
m_generator = MorpheusGenerator(
    spec=m_spec,
    interpreter=m_interpreter,
)

# dumb variable to help infer the shells
m_ps = ProgramSpace(
    m_spec, m_interpreter, [None], None,
)

m_config = {
    "fn":{
        "vocab_size": len(m_ps.get_neighboring_shells())
    },
    "embd_dim": 15*5,
    "meta_test":{
        "n_episode": 250, # only pick the first 250
        "batch_size": 1, # how many attempts
        # "fixed_depth": 4,
        "maxn_attempt": 1000,
        "maxn_step": 3, # program size
        "exploration_rate": 0,
        "benchmarks": "./0731MDsize3.pkl",
    },
}

with open(m_config["meta_test"]["benchmarks"],"rb") as f:
    bmrks = pickle.load(f)

trans_neo = TransNeo(p_config=m_config)
if use_cuda:
    trans_neo = trans_neo.cuda()
optimizer = torch.optim.Adam(list(trans_neo.parameters()))

# writer = SummaryWriter("runs/0713CAMB_RL2_camb3")
writer = None

In [10]:
m_config

{'fn': {'vocab_size': 120},
 'embd_dim': 75,
 'meta_test': {'n_episode': 250,
  'batch_size': 1,
  'maxn_attempt': 1000,
  'maxn_step': 3,
  'exploration_rate': 0,
  'benchmarks': './0731MDsize3.pkl'}}

In [11]:
MetaTest(m_config, m_spec, m_interpreter, m_generator, trans_neo, bmrks, optimizer, writer)

# Start Meta-Test...
# AC/SK/EP:3/10/32, AT:547, SP:2, att.ske.:336.40, att.prog.:114.00,

From cffi callback <function _consolewrite_ex at 0x7f0b629aae18>:
Traceback (most recent call last):
  File "/home/ju-ucsb/anaconda3/lib/python3.7/site-packages/rpy2/rinterface_lib/callbacks.py", line 122, in _consolewrite_ex
    @ffi.callback(WRITECONSOLE_EX_SIGNATURE)
KeyboardInterrupt


# AC/SK/EP:3/10/32, AT:552, SP:0, att.ske.:336.40, att.prog.:114.00,Traceback (most recent call last):
  File "/home/ju-ucsb/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-7c3e2755fa43>", line 1, in <module>
    MetaTest(m_config, m_spec, m_interpreter, m_generator, trans_neo, bmrks, optimizer, writer)
  File "<ipython-input-8-01e66dd3866d>", line 94, in MetaTest
    map_current = p_interpreter.camb_get_shash_abs(var_current)
  File "/home/ju-ucsb/Trinity/MorpheusInterpreter.py", line 904, in camb_get_shash_abs
    df_rhash_val = self.hash(str(self.renv(p_obj))[::-1])%15
  File "/home/ju-ucsb/anaconda3/lib/python3.7/site-packages/rpy2/robjects/__init__.py", line 389, in __call__
    res = self.eval(p)
  File "/home/ju-ucsb/anaconda3/lib/python3.7/site-packages/rpy2/robjects/__init__.py", line 364, in __getattribute__
    return self.__getitem__(attr)
  File "

KeyboardInterrupt: 

In [ ]:
# 3/16/46

In [ ]:
# bmrk_prog, bmrk_str_example = bmrks[0]
# bmrk_example = Example(
#     input=[m_interpreter.load_data_into_var(p) for p in bmrk_str_example.input],
#     output=m_interpreter.load_data_into_var(bmrk_str_example.output),
# )
# ps_solution = ProgramSpace(
#     m_spec, m_interpreter, bmrk_example.input, bmrk_example.output,
# )
# ps_solution.init_by_prog(bmrk_prog)
# # solution_prod_names = [
# #     ps_solution.prod_list[p[0]].name for p in ps_solution.shells
# # ]
# # solution_shells = ps_solution.shells